# Rag From Scratch: Query Transformations

Query transformations are a set of approaches focused on re-writing and / or modifying questions for retrieval.

## Enviornment

`(1) Packages`

In [1]:
#! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

`(2) LangSmith`

https://docs.smith.langchain.com/

In [ ]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = "key"

In [2]:
#! pip install langchain_huggingface

In [ ]:
import getpass
import os

if not os.getenv("HUGGINGFACEHUB_API_TOKEN"):
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = "key"

`(3) API Keys`

## Part 5: Multi Query

Docs:

* https://python.langchain.com/docs/modules/data_connection/retrievers/MultiQueryRetriever

### Index

In [ ]:
#### INDEXING ####

# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

# Index
from langchain_huggingface import HuggingFaceEmbeddings # generates vector embeddings for text
from langchain_community.vectorstores import Chroma
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings_model)

retriever = vectorstore.as_retriever()

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
)

chat_model = ChatHuggingFace(llm=llm)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

### Prompt

In [ ]:
from langchain.prompts import ChatPromptTemplate

# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five
different versions of the given user question to retrieve relevant documents from a vector
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search.
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser

generate_queries = (
    prompt_perspectives
    | chat_model
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

In [ ]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs] # load documents from json to chain

# Retrieve
question = "What is task decomposition for LLM agents?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

6

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

chat_model = ChatHuggingFace(llm=llm)

final_rag_chain = (
    {"context": retrieval_chain,
     "question": itemgetter("question")}
    | prompt
    | chat_model
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'Task decomposition for LLM agents refers to the process of breaking down complex tasks into smaller and simpler subgoals that are more manageable for the LLM to handle. This helps the agent to efficiently handle complicated tasks that would otherwise be overwhelming for the model. Task decomposition can be achieved through techniques like Chain of Thought (CoT) and Tree of Thoughts (TOT) that prompt the LLM to "think step by step" to decompose hard tasks into smaller steps, creating a'

## Part 6: RAG-Fusion

Docs:

* https://github.com/langchain-ai/langchain/blob/master/cookbook/rag_fusion.ipynb?ref=blog.langchain.dev

Blog / repo:

* https://towardsdatascience.com/forget-rag-the-future-is-rag-fusion-1147298d8ad1

### Prompt

In [ ]:
from langchain.prompts import ChatPromptTemplate

# RAG-Fusion: Related
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

In [ ]:
chat_model = ChatHuggingFace(llm=llm)

In [ ]:
from langchain_core.output_parsers import StrOutputParser

generate_queries = (
    prompt_rag_fusion
    | chat_model
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

In [ ]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents
        and an optional parameter k used in the RRF formula
        Algorithm:
        1. Query to different retrievers
        2. Each retriever produces its ranked dcouments
        3. These dicuments combined with RRF formula
        4. Reranking using rrf-scores(sort)
        5. Generate response with top ranked dociments
        """

    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": question})
len(docs)

5

In [ ]:
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""


question = "What is task decomposition for LLM agents?"
prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion,
     "question": itemgetter("question")}
    | prompt
    | chat_model
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

"Task decomposition for LLM agents is the process by which complicated tasks are broken down into smaller, manageable subgoals, enabling efficient handling of complex tasks by the agent. This is achieved through techniques like Chain of Thought (CoT) and Tree of Thoughts, which instruct the LLM to think step by step and generate multiple thoughts per step, shedding light on the model's thinking process. Task decomposition can be done using simple prompts, task-specific instructions, or human"

## Part 7: Decomposition

In [ ]:
from langchain.prompts import ChatPromptTemplate

# Decomposition
template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
Generate multiple search queries related to: {question} \n
Output (3 queries):"""
prompt_decomposition = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain_core.output_parsers import StrOutputParser

chat_model = ChatHuggingFace(llm=llm)

# Chain
generate_queries_decomposition = ( prompt_decomposition | chat_model | StrOutputParser() | (lambda x: x.split("\n")))

# Run
question = "What are the main components of an LLM-powered autonomous agent system?"
questions = generate_queries_decomposition.invoke({"question":question})

In [ ]:
questions

['1. What are the core technologies behind LLM (Large Language Models) and how are they utilized in building autonomous agent systems?',
 '2. What are the key modules or components required for an LLM-powered autonomous agent system, and how do they interact with one another?',
 '3. What challenges emerge in implementing LLM-powered autonomous agent systems, and how are these obstacles being addressed by researchers and engineers?']

### Answer recursively  


Papers:

* https://arxiv.org/pdf/2205.10625.pdf
* https://arxiv.org/abs/2212.10509.pdf

In [ ]:
# Prompt
template = """Here is the question you need to answer:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question:

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

def format_qa_pair(question, answer):
    """Format Q and A pair"""

    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    return formatted_string.strip()

# llm
chat_model = ChatHuggingFace(llm=llm)

q_a_pairs = ""
for q in questions:

    rag_chain = (
    {"context": itemgetter("question") | retriever,
     "question": itemgetter("question"),
     "q_a_pairs": itemgetter("q_a_pairs")}
    | decomposition_prompt
    | chat_model
    | StrOutputParser())

    answer = rag_chain.invoke({"question":q,"q_a_pairs":q_a_pairs})
    q_a_pair = format_qa_pair(q,answer)
    q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair

In [ ]:
answer

'As the field of LLM-powered autonomous agent systems is still emerging, several challenges arise in their implementation. Here are some of the significant obstacles and their current approaches:\n\n1. Data requirements: LLMs demand massive amounts of data to learn and generalize language patterns. Providing such data for open-ended tasks is challenging, as historical data does not always account for contemporary situations. Researchers and engineers are addressing this issue by using synthetic data and pre-trained LLMs to'

In [ ]:
def create_contex(qustions):
    q_a_pairs = ""
    for q in questions:
        rag_chain = (
            {"context": itemgetter("question") | retriever,
                "question": itemgetter("question"),
                "q_a_pairs": itemgetter("q_a_pairs")}
            | decomposition_prompt
            | chat_model
            | StrOutputParser())

        answer = rag_chain.invoke({"question":q,"q_a_pairs":q_a_pairs})
        q_a_pair = format_qa_pair(q,answer)
        q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair
    return q_a_pairs

In [ ]:
template_to_chat = """You are an expert at synthesizing information from question and answer pairs - \n {context} \n
 to form comprehensive and well-structured response. Given the following main question - \n {question} \n
 , along with relevant question-answer pairs, create a clear, informative, and accurate response.

"""
chat_decomposition_prompt = ChatPromptTemplate.from_template(template_to_chat)

In [ ]:
generate_queries_decomposition = ( prompt_decomposition | chat_model | StrOutputParser() | (lambda x: x.split("\n")))

rag_chain = (
    {"context": generate_queries_decomposition | create_contex,
     "question": itemgetter("question")}
    | chat_decomposition_prompt
    | chat_model
    | StrOutputParser()
)

rag_chain.invoke({"question": question})

'The key components required for an LLM-powered autonomous agent system, as outlined in the given background materials and further explained through related question-answer pairs, include a Large Language Model (LLM) as the core controller and brain of the agent, responsible for understanding and processing natural language inputs and generating appropriate responses. The system also requires planning components such as task decomposition and goal selection, as well as execution modules for carrying out tasks. Additionally, feedback and learning mechanisms are essential to enable agents to'

In [ ]:
question

'What are the main components of an LLM-powered autonomous agent system?'

### Answer individually


In [ ]:
# Answer each sub-question individually

from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

# RAG prompt
prompt_rag = hub.pull("rlm/rag-prompt")

chat_model = ChatHuggingFace(llm=llm)

def retrieve_and_rag(question,prompt_rag,sub_question_generator_chain):
    """RAG on each sub-question"""

    # Use our decomposition /
    sub_questions = sub_question_generator_chain.invoke({"question":question})

    # Initialize a list to hold RAG chain results
    rag_results = [sub_questions]

    for sub_question in sub_questions:

        # Retrieve documents for each sub-question
        retrieved_docs = retriever.get_relevant_documents(sub_question)

        # Use retrieved documents and sub-question in RAG chain
        answer = (prompt_rag | chat_model | StrOutputParser()).invoke({"context": retrieved_docs,
                                                                "question": sub_question})
        rag_results.append(answer)
    return rag_results

# Wrap the retrieval and RAG process in a RunnableLambda for integration into a chain
answers_and_questions = retrieve_and_rag(question, prompt_rag, generate_queries_decomposition)

In [ ]:
def format_qa_pairs(data):
    """Format Q and A pairs"""
    questions, answers = data[0], data[1:]
    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
    return formatted_string.strip()

context = format_qa_pairs(answers_and_questions)

# Prompt
template = """Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | chat_model
    | StrOutputParser()
)

final_rag_chain.invoke({"context":context,"question":question})

"Task decomposition for LLM (Large Language Models) agents refers to the process of breaking down complex tasks into smaller, manageable subgoals. This allows the agents to handle complex tasks more efficiently and with a clear path to follow. CoT (Chain of Thought) and Tree of Thoughts are techniques used for task decomposition, helping the agent transform big tasks into multiple manageable tasks and shed light on the model's thinking process. Subgoal and decomposition assist the agent in handling complex"

In [ ]:
template = """Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
answers_and_questions = retrieve_and_rag(question, prompt_rag, generate_queries_decomposition)
context = format_qa_pairs(answers_and_questions)

rag_chain = (
    {"context": RunnableLambda(lambda x: context),
     "question": itemgetter("question")}
    | prompt
    | chat_model
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke({"question":question})

"Task decomposition for LLM (Large Language Models) agents refers to the process of breaking down complex tasks into smaller, manageable subgoals. This allows the agents to handle complex tasks more efficiently and with a clear path to follow. CoT (Chain of Thought) and Tree of Thoughts are techniques used for task decomposition, helping the agent transform big tasks into multiple manageable tasks and shed light on the model's thinking process. Subgoal and decomposition assist the agent in handling complex"

## Part 8: Step Back

Paper:

* https://arxiv.org/pdf/2310.06117.pdf

In [ ]:
# Few Shot Examples
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]
# We now transform these to example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        # Few shot examples
        few_shot_prompt,
        # New question
        ("user", "{question}"),
    ]
)

In [ ]:
generate_queries_step_back = prompt | chat_model | StrOutputParser()
question = "What is task decomposition for LLM agents?"
generate_queries_step_back.invoke({"question": question})

'How can language model (LLM) agents be divided into smaller parts to perform tasks more efficiently?'

In [ ]:
from operator import itemgetter
# Response prompt
response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)
chain = (
    {
        # Retrieve context using the normal question
        "normal_context": itemgetter('question') | retriever,
        # Retrieve context using the step-back question
        "step_back_context": generate_queries_step_back | retriever,
        # Pass on the question
        "question": itemgetter('question'),
    }
    | response_prompt
    | chat_model
    | StrOutputParser()
)

chain.invoke({"question": question})

'Task decomposition is a key component in LLM-powered autonomous agents, which enables them to handle complex tasks efficiently. It breaks large tasks into smaller, manageable subgoals, allowing the agent to plan ahead and make progress in a systematic way.\n\nThis process involves two main steps: subgoal and decomposition. First, the agent identifies the subgoals required to complete the overall task. This can be done through simple prompting or by using task-specific instructions. The'

## Part 9: HyDE

Docs:

* https://github.com/langchain-ai/langchain/blob/master/cookbook/hypothetical_document_embeddings.ipynb

Paper:

* https://arxiv.org/abs/2212.10496

In [ ]:
from langchain.prompts import ChatPromptTemplate

# HyDE document genration
template = """Please write a scientific paper passage to answer the question
Question: {question}
Passage:"""
prompt_hyde = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser

generate_docs_for_retrieval = (
    prompt_hyde | chat_model | StrOutputParser()
)

# Run
question = "What is task decomposition for LLM agents?"
generate_docs_for_retrieval.invoke({"question":question})

'Task decomposition refers to the process of breaking down complex tasks into smaller subtasks that are more manageable and solvable for a large language model (LLM) agent. In the context of LLMs, subtasks may involve specific types of reasoning, such as question answering, text classification, or syntactic parsing, and may be performed concurrently or in sequence. The use of task decomposition allows LLMs to more efficiently learn and perform heterogeneous tasks, while improving their overall performance and'

In [ ]:
# Retrieve
retrieval_chain = generate_docs_for_retrieval | retriever
retireved_docs = retrieval_chain.invoke({"question":question})
retireved_docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search

In [ ]:
# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":retireved_docs,"question":question})

'\nAnswer: Task decomposition for LLM agents involves breaking down complex tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks. This process is essential because a complicated task usually involves many steps, and an agent needs to know what they are and plan ahead. Techniques like Chain of Thought (CoT) and Tree of Thoughts have become standard prompting techniques for enhancing model performance on complex tasks by decomposing hard tasks into smaller and simpler steps. These techniques shed light on the agent\'s thinking process and provide insights into the interpretation of the model\'s performance on complex tasks. Task decomposition can be done using simple prompts like "Steps for XYZ." or "What are the subgoals for achieving XYZ?" or with human inputs or task-specific instructions. However, challenges in long-term planning and task decomposition remain, as LLMs struggle to adjust plans when faced with unexpected errors, making them less robust 

In [ ]:
from operator import itemgetter

template = """Please write a scientific paper passage to answer the question
Question: {question}
Passage:"""
prompt_hyde = ChatPromptTemplate.from_template(template)

final_template = """Answer the following question based on this context:

{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(final_template)
final_rag_chain = (
    {'context': prompt_hyde | chat_model | StrOutputParser() | retriever,
     'question': itemgetter('question')}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
final_rag_chain.invoke({'question': question})

'\nAnswer: Task decomposition for LLM agents involves breaking down complex tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks. This process is essential because a complicated task usually involves many steps, and an agent needs to know what they are and plan ahead. Techniques like Chain of Thought (CoT) and Tree of Thoughts have become standard prompting techniques for enhancing model performance on complex tasks by decomposing hard tasks into smaller and simpler steps. These techniques shed light on the agent\'s thinking process and provide insights into the interpretation of the model\'s performance on complex tasks. Task decomposition can be done using simple prompts like "Steps for XYZ." or "What are the subgoals for achieving XYZ?" or with human inputs or task-specific instructions. However, challenges in long-term planning and task decomposition remain, as LLMs struggle to adjust plans when faced with unexpected errors, making them less robust 